## `address_cleaner`: create consistent addresses that maximize success in geocoding.

### What it does:
#### Corrections
* Consistent city names
* Consistent street name abbreviations, specifically type and direction

#### Flags
* Street names missing
* Intersections in place of street addresses
* Building numbers equal to 0
* Building numbers missing
* Building number ranges

### What you do:
Edit the two lines below as paths to your existing import `.csv` and your desired export `.csv`.

### Cautions:
* This program assumes you've run your addresses through the parsing scripts (see `parse_nets.ipynb` and `parse_costar.ipynb`), which export `.csvs` in an identical format. Tossing in datasets other than these will almost certainly yield failure.
* This program is written to address idiosyncracies of the Southeastern PA region (e.g. "Street Road" and city names such as "Mount Laurel"). While it can be modified for other locations, its full functionality is not immediately portable.

In [1]:
import_file = "parse_costar.csv"
export_file = "clean_costar.csv"

### Should not have to edit anything below here.

In [2]:
import re, csv, yaml

In [3]:
# Initial upload fields
ids = []
nums = []
roads = []
cities = []
states = []
zips = []
buildings = []
units = []
levels = []
neighborhoods = []
# Fields populated through cleaning
zero_start_flag = []
no_number_flag = []
intersection_flag = []
no_name_flag = []
hyphen_flag = []

# Read .csv
with open(import_file) as io:
    r = csv.reader(io, delimiter = ",")
    next(r, None) # Ignore column names
    for row in r:
        row_id = row[0]
        row_num = row[1]
        row_road = row[2]
        row_city = row[3]
        row_state = row[5]
        row_zip = row[7]
        row_building = row[8]
        row_unit = row[9]
        row_level = row[10]
        row_neighborhood = row[13]
        ids.append(row_id)
        nums.append(row_num)
        roads.append(row_road)
        cities.append(row_city)
        states.append(row_state)
        zips.append(row_zip)
        buildings.append(row_building)
        units.append(row_unit)
        levels.append(row_level)
        neighborhoods.append(row_neighborhood)

### Consistent city names
Replace city name abbreviations with full name. So far, the only potential issues with city names have been in Kennett Square and Mount Laurel. Will need to be added to if other inconsistencies are found.

In [4]:
i = 0
while (i < len(cities)):
    cities[i] = re.sub(r'\ssq[\W]?$', ' square', cities[i], flags = re.IGNORECASE)
    cities[i] = re.sub(r'^mt[\W]?\s', 'mount ', cities[i], flags = re.IGNORECASE)
    i = i + 1

### Consistent street name abbreviations
Use OpenCage's English-language road abbreviations, [`en.yaml`](https://github.com/OpenCageData/address-formatting/blob/master/conf/abbreviations/en.yaml), to identify these.
* Our region has a "Street Road" which should not abbreviated to "st rd". For this reason, instances of "Street Road" are ignored before standardizing abbreviations of "street" in the beginning or middle of the street name string.
* Because directions (`N, NE...`) can show up anywhere in an address (e.g. "Northampton"), expressions only replace individual instances of pre- and post-modifiers separated from other words in the street name by spaces.
* To make corrections easier later, here are `regex` commands to identify and sub the following:
  * Beginning: `re.sub(r'^north\s|^n[\W]\s', 'n ', 'north 14th street', flags = re.IGNORECASE)`
  * Middle: `re.sub(r'^savenue\s|\save[\W]\s', ' ave ', 'park avenue west', flags = re.IGNORECASE)`
  * End: `re.sub(r'\snorth$|\sn[\W]$', ' n', 'lightcap road north', flags = re.IGNORECASE)`
* "Terrace" sub for middle of string is currently commented out, since there is a "Terrace Avenue." Other street names may require special treatment if they appear often enough.

In [5]:
street_road = re.compile(r'street\sroad|street\srd[\W]?')
with open("en.yaml", 'r') as stream:
    try:
        street_key = yaml.load(stream)
        street_key = street_key["road"]
        print(street_key)
    except yaml.YAMLError as exc:
        print(exc)

i = 0
while (i < len(roads)):
    # Pre-type + pre-directional
    roads[i] = re.sub(r'^alley\s|^aly[\W]\s', 'aly ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^arcade\s|^arc[\W]\s', 'arc ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^avenue\s|^ave[\W]\s', 'ave ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^boulevard\s|^blvd[\W]\s', 'blvd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^circle\s|^cl[\W]\s', 'cl ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^court\s|^ct[\W]\s', 'ct ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^crescent\s|^cres[\W]\s', 'cres ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^crossroad\s|^xrd[\W]\s', 'xrd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^drive\s|^dr[\W]\s', 'dr ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^esplanade\s|^esp[\W]\s', 'esp ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^expressway\s|^expy[\W]\s', 'expy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^extension\s|^ext[\W]\s', 'ext ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^freeway\s|^fwy[\W]\s', 'fwy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^grove\s|^gr[\W]\s', 'gr ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^highway\s|^hwy[\W]\s', 'hwy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^mountain\s|^mtn[\W]\s', 'mtn ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^place\s|^pl[\W]\s', 'pl ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^road\s|^rd[\W]\s', 'rd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^square\s|^sq[\W]\s', 'sq ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^terrace\s|^tce[\W]\s', 'tce ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^throughway\s|^trwy[\W]\s', 'trwy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^north\s|^n[\W]', 'n ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^south\s|^s[\W]', 's ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^east\s|^e[\W]', 'e ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^west\s|^w[\W]', 'w ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^northeast\s|^ne[\W]', 'ne ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^southeast\sse|^n[\W]', 'se ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^northwest\s|^nw[\W]', 'nw ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'^southwest\s|^sw[\W]', 'sw ', roads[i], flags = re.IGNORECASE)
    
    # Type in middle of string
    roads[i] = re.sub(r'\salley\s|\saly[\W]\s', ' aly ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sarcade\s|\sarc[\W]\s', ' arc ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\savenue\s|\save[\W]\s', ' ave ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sboulevard\s|\sblvd[\W]\s', ' blvd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scircle\s|\scl[\W]\s', ' cl ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scourt\s|\sct[\W]\s', ' ct ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\screscent\s|\scres[\W]\s', ' cres ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scrossroad\s|\sxrd[\W]\s', ' xrd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sdrive\s|\sdr[\W]\s', ' dr ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sesplanade\s|\sesp[\W]\s', ' esp ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sexpressway\s|\sexpy[\W]\s', ' expy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sextension\s|\sext[\W]\s', ' ext ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sfreeway\s|\sfwy[\W]\s', ' fwy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sgrove\s|\sgr[\W]\s', ' gr ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\shighway\s|\shwy[\W]\s', ' hwy ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\smountain\s|\smtn[\W]\s', ' mtn ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\splace\s|\spl[\W]\s', ' pl ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sroad\s|\srd[\W]\s', ' rd ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\ssquare\s|\ssq[\W]\s', ' sq ', roads[i], flags = re.IGNORECASE)
    # roads[i] = re.sub(r'\sterrace\s|\stce[\W]\s', ' tce ', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sthroughway\s|\strwy[\W]\s', ' trwy ', roads[i], flags = re.IGNORECASE)
    
    # Post-type + post-directional
    roads[i] = re.sub(r'\salley$|\saly[\W]$', ' aly', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sarcade$|\sarc[\W]$', ' arc', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\savenue$|\save[\W]$', ' ave', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sboulevard$|\sblvd[\W]$', ' blvd', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scircle$|\scl[\W]$', ' cl', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scourt$|\sct[\W]$', ' ct', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\screscent$|\scres[\W]$', ' cres', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\scrossroad$|\sxrd[\W]$', ' xrd', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sdrive$|\sdr[\W]$', ' dr', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sesplanade$|\sesp[\W]$', ' esp', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sexpressway$|\sexpy[\W]$', ' expy', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sextension$|\sext[\W]$', ' ext', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sfreeway$|\sfwy[\W]$', ' fwy', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sgrove$|\sgr[\W]$', ' gr', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\shighway$|\shwy[\W]$', ' hwy', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\smountain$|\smtn[\W]$', ' mtn', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\splace$|\spl[\W]$', ' pl', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sroad$|\srd[\W]$', ' rd', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\ssquare$|\ssq[\W]$', ' sq', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sstreet$|\sst[\W]$', ' st', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sterrace$|\stce[\W]$', ' tce', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\sthroughway$|\strwy[\W]$', ' trwy', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\snorth$|\sn[\W]$', ' n', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\ssouth$|\ss[\W]$', ' s', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\seast$|\se[\W]$', ' e', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\swest$|\sw[\W]$', ' w', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\snortheast$|\sne[\W]$', ' ne', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\ssoutheast$|\sse[\W]$', ' se', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\snorthwest$|\snw[\W]$', ' nw', roads[i], flags = re.IGNORECASE)
    roads[i] = re.sub(r'\ssouthwest$|\ssw[\W]$', ' sw', roads[i], flags = re.IGNORECASE)
    
    if bool(street_road.search(str(roads[i]))) == False:
        roads[i] = re.sub(r'^street\s|^st[\W]\s', 'st ', roads[i], flags = re.IGNORECASE)
        roads[i] = re.sub(r'\sstreet\s|\sst[\W]\s', ' st ', roads[i], flags = re.IGNORECASE)
    
    i = i + 1

{'Alley': 'Aly', 'Arcade': 'Arc', 'Avenue': 'Ave', 'Boulevard': 'Blvd', 'Circle': 'Cl', 'Court': 'Ct', 'Crescent': 'Cres', 'Crossroad': 'XRD', 'Drive': 'Dr', 'Esplanade': 'Esp', 'Expressway': 'EXPY', 'Extention': 'Ext', 'Freeway': 'Fwy', 'Grove': 'Gr', 'Highway': 'HWY', 'Mountain': 'Mtn', 'Northeast': 'NE', 'Northwest': 'NW', 'Place': 'Pl', 'Road': 'Rd', 'Southeast': 'SE', 'Southwest': 'SW', 'Square': 'Sq', 'Street': 'St', 'Terrace': 'Tce', 'Throughway': 'TRWY'}


### Street names missing
Flag addresses with no street name. List `no_name_flag` is a Boolean flag indicating missing street names.

In [6]:
no_start = re.compile(r'^$')
i = 0
while (i < len(nums)):
    no_name_item = bool(no_start.search(str(roads[i])))
    no_name_item = no_name_item * 1
    no_name_flag.append(no_name_item)
    i = i + 1

### Intersections in place of street addresses
Flag for manual correction. List `intersection_flag` is a Boolean flag indicating if the street name contains `@`, `&`, or `and`.

In [7]:
intersection = re.compile(r'\sand\s|\s&\s|\s@\s')
i = 0
while (i < len(roads)):
    intersection_item = bool(intersection.search(str(roads[i])))
    intersection_item = intersection_item * 1
    intersection_flag.append(intersection_item)
    i = i + 1

### Building numbers equal to 0
Eliminate leading zeroes. Function `num_test` converts to numeric when possible and leaves as string when not. List `zero_start_flag` is a Boolean flag indicating if building number equals zero. This may export as `True` and `False`: in this case, edits need to be made as the variable is created.

In [8]:
def num_test(s):
    try:
        return int(s)
    except ValueError:
        return str(s)
i = 0
while (i < len(nums)):
    nums[i] = num_test(nums[i])
    i = i + 1
zero_start = re.compile(r'^[0]')
i = 0
while (i < len(nums)):
    zero_start_item = bool(zero_start.search(str(nums[i])))
    zero_start_item = zero_start_item * 1
    zero_start_flag.append(zero_start_item)
    i = i + 1

### Building numbers missing
Flag addresses with no building number. List `no_number_flag` is a Boolean flag indicating if the building has no number.

In [9]:
no_start = re.compile(r'^$')
i = 0
while (i < len(nums)):
    no_number_item = bool(no_start.search(str(nums[i])))
    no_number_item = no_number_item * 1
    no_number_flag.append(no_number_item)
    i = i + 1

### Building number ranges
Flag addresses with hyphens in the street number. List `hyphen_flag` is a Boolean flag indicating the presence of hyphens.

In [10]:
hyphen = re.compile(r'-')
i = 0
while (i < len(nums)):
    hyphen_item = bool(hyphen.search(str(nums[i])))
    hyphen_item = hyphen_item * 1
    hyphen_flag.append(hyphen_item)
    i = i + 1

### Export

In [11]:
l = [ids, nums, roads, cities, states, zips, buildings, units, levels, neighborhoods,
     zero_start_flag, no_number_flag, intersection_flag, no_name_flag, hyphen_flag]
reconstitute = zip(*l)

with open(export_file, "w", newline="") as io:
    writer = csv.writer(io, dialect = "excel")
    writer.writerow(["id", "house_number", "road", "city", "state", "zip",
                     "building", "unit", "level", "neighborhood", "zero_number_flag",
                     "no_number_flag", "intersection_flag", "no_name_flag", "hyphen_flag"])
    writer.writerows(reconstitute)
io.close()